<a href="https://colab.research.google.com/github/cypision/Alchemy-in-MLDL/blob/master/CNN_sentimental_analysis_naver_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 준비단계 - 필요 라이브러리 설치 등
네이버 영화 리뷰 data는 별도로 제공받은 파일(출처공개 불가. 죄송 ㅜ.ㅜ)을 사용합니다.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install konlpy
!pip install jpype1==0.7.0

     |████████████████████████████████| 19.4MB 73.5MB/s 
     |████████████████████████████████| 460kB 51.6MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 2.7MB 7.5MB/s 
  Found existing installation: JPype1 1.2.0
    Uninstalling JPype1-1.2.0:
      Successfully uninstalled JPype1-1.2.0


In [1]:
import numpy as np
import json
import random
import pandas as pd
import tensorflow as tf

In [2]:
# from nltk.corpus import stopwords : 한국어는 직접 불용어를 설정해야만 한다.

In [3]:
with open("/content/gdrive/My Drive/CNS_NLP/Sentiment_train.json") as f:
  train = json.loads(f.read())
with open("/content/gdrive/My Drive/CNS_NLP/Sentiment_val.json") as f:
  val = json.loads(f.read())
with open("/content/gdrive/My Drive/CNS_NLP/Sentiment_test.json") as f:
  test = json.loads(f.read())

In [4]:
## 형태소 분석기 불러오기
from konlpy.tag import Komoran, Hannanum, Kkma, Okt ## using only Komoran object

## 형태소분석 함수 만들기
komoran = Komoran()
def tokenize(word):
  return komoran.morphs(word)

In [5]:
print(test[0])
print(test[0][1])
tokenize(test[0][1])

['1458790', '허우 샤오시엔 작품은 모두 만점!', '긍정']
허우 샤오시엔 작품은 모두 만점!


['허', '우', '샤오', '시엔', '작품', '은', '모두', '만점', '!']

In [6]:
## train,val,test 따로 떨어져 있으니 이를 한꺼번에 합친이후 단어사전을 만든다.
print(len(train),len(val),len(test))
total_setence = []

total_setence.extend(train)
total_setence.extend(val)
total_setence.extend(test)

print(len(total_setence))

50000 10000 10000
70000


In [8]:
total_setence[11]

['6950777', '역시 미셸 오슬로네요~ 이 작품은 꼭 극장에서 큰 스크린으로 봐야할듯해요~ 한번 더봐야지~*', '긍정']

In [9]:
## label 구성보기  
label = []
for lst in total_setence:
  label.append(lst[2])

label = set(label)
print(label)

{'부정', '긍정'}


In [10]:
list(label)[0]

'부정'

In [11]:
label_map = {list(label)[0]:0,list(label)[1]:1}
print(label_map)

{'부정': 0, '긍정': 1}


In [12]:
## train set 에서 단어사전 만들기  (불용어 처리 없이 간다.)
import collections 
from tqdm import tqdm

tot_tokens = 0
komoran_counter = collections.Counter() # 카운터

for dat in tqdm(total_setence):
  sent = dat[1] ## real setence
  tokenized_sent = tokenize(sent)
  for cha in tokenized_sent:
    komoran_counter[cha] += 1

100%|██████████| 70000/70000 [00:53<00:00, 1315.17it/s]


In [18]:
print(len(komoran_counter))
tokens = komoran_counter.most_common()
print(type(tokens),type(tokens[0]))

38724
<class 'list'> <class 'tuple'>


In [21]:
all_token = [to[0] for to in tokens]
vocab_list = ["[PAD]","[UNK]"]
vocab_list.extend(all_token)

In [23]:
vocab_list[190]

'역시'

In [24]:
##  불용어 처리 없이, index화 합니다. 이때, max_len 을 지정해서, 최종적으로 train,val,test 데이터 셋을 학습가능토록 형태를 만듭니다.  
char_to_idx = {}
idx_to_char = {}

for i, token in enumerate(vocab_list):
  char_to_idx[token] = i

for key,value in char_to_idx.items():
  idx_to_char[value] = key 

In [30]:
print(vocab_list[190])
print(char_to_idx[vocab_list[190]])
print(idx_to_char[char_to_idx[vocab_list[190]]])

역시
190
역시


**단어사전이 완성되었습니다.**  
상기과정에 [PAD],[UNK] 를 추가하여, 클래스로 만든것이 TextEncoder 입니다.  
하기에서는 그 class를 불러서 만들겠습니다.  
class가 하는 내용은 상기과정과 똑같지만, 깔끔하게 만들기 위해 class화 했습니다.

In [31]:
!cp "/content/gdrive/My Drive/CNS_NLP/utils.py" "/content"

In [32]:
from utils import TextEncoder

In [33]:
print(len(vocab_list))
print("# Vocabs = {}".format(len(vocab_list)))

38726
# Vocabs = 38726


In [34]:
vocab_list[190]

'역시'

In [35]:
text_encoder = TextEncoder(vocab_list)

In [ ]:
tokenize('역시 미셸 오슬로네요~ 이 작품은 꼭 극장에서 큰 스크린으로 봐야할듯해요~ 한번 더봐야지~*')

In [45]:
print(train[11])
z = train[11][1]
rslt = text_encoder.convert_tokens_to_ids(z)
print(rslt[0:10])

['6950777', '역시 미셸 오슬로네요~ 이 작품은 꼭 극장에서 큰 스크린으로 봐야할듯해요~ 한번 더봐야지~*', '긍정']
[765, 89, 1, 440, 1, 1, 110, 1208, 58, 60]


In [46]:
z

'역시 미셸 오슬로네요~ 이 작품은 꼭 극장에서 큰 스크린으로 봐야할듯해요~ 한번 더봐야지~*'

## 상기 코드에서 이상한점이 없는가?  z 는 문장인데? (필자가 틀린부분이라 크게 기록했다) ㅋ  
이상한 점을 찾기 위해 아래 for 문을 보자

In [49]:
for i in z :
  print(i)

역
시
 
미
셸
 
오
슬
로
네
요
~
 
이
 
작
품
은
 
꼭
 
극
장
에
서
 
큰
 
스
크
린
으
로
 
봐
야
할
듯
해
요
~
 
한
번
 
더
봐
야
지
~
*


**그렇다. z 는 tokenize 되지 않는 그냥 문장이다. 이걸 적용하면, String Object로 인식하고 이 클래스는 기본적으로 한글자,한글자 ASCII 코드로 인식, 공백과 , 까지 인색하여 convert_to_idx 한다. 즉 형태소 형태로 분해되지 않은채 코딩되는 것이다. 이 부분을 주의해야 한다.**


In [50]:
train[11]

['6950777', '역시 미셸 오슬로네요~ 이 작품은 꼭 극장에서 큰 스크린으로 봐야할듯해요~ 한번 더봐야지~*', '긍정']

In [57]:
tokenize(train[11][2])

['긍정']

In [58]:
## train,val,test 를 정수형 index 벡터로 바꾸기 전에, 먼저 한덩어리로 되어 있는 setence를 tokenize해야 한다.
tokenize_train = []
tokenize_val = []
tokenize_test = []

for dat in tqdm(train):
  sent = dat[1] ## real setence
  tokenized_sentence = tokenize(sent)
  rslt1, rslt2, rslt3 = dat[0],tokenized_sentence,dat[2]
  tokenize_train.append([rslt1, rslt2, rslt3])

for dat in tqdm(val):
  sent = dat[1] ## real setence
  tokenized_sentence = tokenize(sent)
  rslt1, rslt2, rslt3 = dat[0],tokenized_sentence,dat[2]
  tokenize_val.append([rslt1, rslt2, rslt3])

for dat in tqdm(test):
  sent = dat[1] ## real setence
  tokenized_sentence = tokenize(sent)
  rslt1, rslt2, rslt3 = dat[0],tokenized_sentence,dat[2]
  tokenize_test.append([rslt1, rslt2, rslt3])

100%|██████████| 10000/10000 [00:07<00:00, 1363.16it/s]


In [59]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [60]:
label_map

{'긍정': 1, '부정': 0}

In [61]:
## text_encoder를 활용하여, 정수 index 구성으로 바꾸기
train_input_lst = []
val_input_lst = []
test_input_lst = []

train_label = []
val_label = []
test_label = []

for data in tokenize_train :
  setence = data[1]
  tokenized_idx_lst = text_encoder.convert_tokens_to_ids(setence)
  train_input_lst.append(tokenized_idx_lst) ## 2중 list 형태로 데이터를 만든다.
  train_label.append(label_map[data[2]])

for data in tokenize_val :
  setence = data[1]
  tokenized_idx_lst = text_encoder.convert_tokens_to_ids(setence)
  val_input_lst.append(tokenized_idx_lst) ## 2중 list 형태로 데이터를 만든다.
  val_label.append(label_map[data[2]])

for data in tokenize_test :
  setence = data[1]
  tokenized_idx_lst = text_encoder.convert_tokens_to_ids(setence)
  test_input_lst.append(tokenized_idx_lst) ## 2중 list 형태로 데이터를 만든다.  
  test_label.append(label_map[data[2]])

In [63]:
setence00 = train[11][1]
print(setence00)
setence01 = tokenize_train[11][1]
print(setence01)

setence02 = text_encoder.convert_tokens_to_ids(setence01)
print(setence02)

역시 미셸 오슬로네요~ 이 작품은 꼭 극장에서 큰 스크린으로 봐야할듯해요~ 한번 더봐야지~*
['역시', '미셸', '오슬로', '네', '요', '~', '이', '작품', '은', '꼭', '극장', '에서', '크', 'ㄴ', '스크린', '으로', '보', '아야', '하', 'ㄹ', '듯', '하', '어요', '~', '한', '번', '더', '보', '아야지', '~', '*']
[190, 8891, 10379, 60, 135, 33, 3, 128, 14, 220, 352, 57, 280, 5, 1708, 45, 9, 162, 4, 21, 114, 4, 53, 33, 94, 134, 96, 9, 683, 33, 632]


In [64]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [65]:
## 길이가 일정한 문장으로 만들기  
## """ max_seq_len을 넘는 문장은 절단, 모자르는 것은 PADDING """

# input_ids = pad_sequences(input_ids,maxlen=max_seq_len,padding="post",truncating="pre")

max_seq_len = 150
train_ids = pad_sequences(train_input_lst,maxlen=max_seq_len,padding="post",truncating="pre")
val_ids = pad_sequences(val_input_lst,maxlen=max_seq_len,padding="post",truncating="pre")
test_ids = pad_sequences(test_input_lst,maxlen=max_seq_len,padding="post",truncating="pre")

In [69]:
print(train[11])
print("="*500)
print(train_input_lst[11],"\n seuquence before: {}".format(len(train_input_lst[11])))
print("="*500)
print(train_ids[11],"\n seuquence after: {}".format(len(train_ids[11])))

['6950777', '역시 미셸 오슬로네요~ 이 작품은 꼭 극장에서 큰 스크린으로 봐야할듯해요~ 한번 더봐야지~*', '긍정']
[190, 8891, 10379, 60, 135, 33, 3, 128, 14, 220, 352, 57, 280, 5, 1708, 45, 9, 162, 4, 21, 114, 4, 53, 33, 94, 134, 96, 9, 683, 33, 632] 
 seuquence before: 31
[  190  8891 10379    60   135    33     3   128    14   220   352    57
   280     5  1708    45     9   162     4    21   114     4    53    33
    94   134    96     9   683    33   632     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0 

In [70]:
train_label[11]

1

In [67]:
type(val_label)

list

In [68]:
## 데이터 저장하기
## train_input_lst,train_label,val_input_lst,val_label,test_input_lst,test_label
## train_ids 를 저장하고 싶었으나, numpy의 경우, json 파일로 저장이 되지 않기 때문에 통일성을 위해, 그냥 pad sequence 이전것을 저장한다.  
with open('/content/gdrive/My Drive/CNS_NLP/naver_m_train_lst.json', 'w',encoding='utf-8') as f:
  json.dump(train_input_lst, f,indent="\t")
with open('/content/gdrive/My Drive/CNS_NLP/naver_m_train_label.json', 'w',encoding='utf-8') as f:
  json.dump(train_label, f,indent="\t")
with open('/content/gdrive/My Drive/CNS_NLP/naver_m_val_lst.json', 'w',encoding='utf-8') as f:
  json.dump(val_input_lst, f,indent="\t")
with open('/content/gdrive/My Drive/CNS_NLP/naver_m_val_label.json', 'w',encoding='utf-8') as f:
  json.dump(val_label, f,indent="\t")
with open('/content/gdrive/My Drive/CNS_NLP/naver_m_test_lst.json', 'w',encoding='utf-8') as f:
  json.dump(test_input_lst, f,indent="\t")
with open('/content/gdrive/My Drive/CNS_NLP/naver_m_test_label.json', 'w',encoding='utf-8') as f:
  json.dump(test_label, f,indent="\t")

**이후 과정은 별도 posting으로 운영한다.**